In [1]:
import os
from pprint import pprint
import pandas as pd
from typing import Any
import boto3
import requests
from dotenv import load_dotenv
from scrolling import wrapping_data

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [6]:
load_dotenv()

print(boto3.__version__)

client_id = os.getenv("uat_client_id")
username = os.getenv("username")
password = os.getenv("password")
uat_url = os.getenv("uat_url")
get_url = os.getenv("get_url")
post_url = os.getenv("post_url")

1.28.15


In [17]:
class VVIPReporting:
    """Class that generates VVIP reporting"""

    def __init__(self, url: str, get: str, post: str) -> None:
        self.client_id = client_id
        self.username = username
        self.password = password
        self.url = url
        self.get_url = get
        self.post_url = post
        id_token = self._authenticate()
        # print(id_token)
        raw_data = self._get_data(id_token)
        self._get_report(raw_data)

    def _authenticate(self) -> str:
        client = boto3.client("cognito-idp", region_name="eu-west-1")

        # Initiating the Authentication,
        response = client.initiate_auth(
            ClientId=self.client_id,
            AuthFlow="USER_PASSWORD_AUTH",
            AuthParameters={
                "USERNAME": self.username,
                "PASSWORD": self.password,
            },
        )

        return response["AuthenticationResult"]["IdToken"]

    def _get_data(self, id_token: str) -> pd.DataFrame:
        resp = requests.get(
            url=self.url + self.get_url,
            headers={
                "Authorization": f"Bearer {id_token}",
            },
            timeout=30,
        )
        # pprint(resp.json(), indent=2)
        return pd.DataFrame.from_dict(resp.json()["Items"])

    def _get_report(self, data: pd.DataFrame) -> pd.DataFrame:
        data.set_index("Customer_id", inplace=True)
        #print(data.head(10))
        print(data.shape)

        # converting from isoformat to datetime
        iso_lists = ["Subscription_date", "Expiration_date", "Renewal_date"]
        for date_col in iso_lists:
            data[date_col] = pd.to_datetime(
                data[date_col], format="ISO8601", errors="coerce"
            ).dt.date
            # data.sort_index("Subscription_date", inplace=True, ascending=True)
        data.sort_values(by="Subscription_date", inplace=True, ascending=True)
        # visualize dataframe in a nice format
        return wrapping_data(data)

In [18]:
VVIPReporting(uat_url, get_url, post_url)

(46, 14)


,Subscription_date,Renewal_date,Expiration_date,UserGroup,SalesChannel,Price,SalesPerson,SalesPerson_flag,Customer_flag,Coupon,Company,Username,Name,Surname
Customer_id,,,,,,,,,,,,,,
A-8ccb0f02-dab7-42f5-b6aa-881b39de9735,2023-05-11,NaT,NaT,none,none,none,none,inactive,Admin,none,none,j.a.f.colombo@gmail.com,Jacopo,Colombo
A-01f97264-b43a-4ff4-8ad7-e48fea52a3e8,2023-05-11,NaT,NaT,none,none,none,none,inactive,Admin,none,none,gavrila@formulacoach.it,Test,Test
A-b957b10d-e0c7-4777-8952-bb763008989b,2023-05-11,NaT,NaT,none,none,none,none,inactive,Admin,none,none,giordano@formulacoach.it,andrea,giordano
F-49ffb362-977d-49fd-af1a-45e810ffda50,2023-05-17,NaT,2023-06-17,none,Online,none,none,inactive,Free Trial,13358457-1F24-4667-BDAC-72F624E20616,Microsoft,vvipuser41+uat_test@gmail.com,Bill,Gates
A-2f2db8d0-39c8-46ea-9ad2-9ae5ab4ec10f,2023-05-18,NaT,NaT,none,none,none,none,inactive,Admin,none,none,darkfenner69@gmail.com,Raimondo,Marino
F-3d7d76de-07f5-47c4-b26a-d749a52108f4,2023-06-28,NaT,2023-08-07,none,Online,none,none,inactive,Free Trial,8CD29D30-BB88-4A16-9C91-F50898A4F106,Test,test@yopmail.com,test,test
F-f09323f4-64d0-4b82-8423-4b022feb88fb,2023-06-28,NaT,2023-08-07,none,Online,none,none,inactive,Free Trial,8CD29D30-BB88-4A16-9C91-F50898A4F106,test2,test2@yopmail.com,test2,test2
F-4133ba11-37f2-4ce0-91d1-0aabf865fb9e,2023-06-29,NaT,2023-08-08,none,Online,none,none,inactive,Free Trial,8CD29D30-BB88-4A16-9C91-F50898A4F106,OpenAI,altman2@yopmail.com,Sam,Altman
F-ccf887fc-951f-4972-937a-71e5a194bc1e,2023-07-21,NaT,2023-07-21,none,DirectSales,none,ceravolo@formulacoach.it,active,Free Trial,30093A9E-AF40-4E7B-BDD9-CEC64FD5374C,none,vvipuser41+test_email@gmail.com,test_email,test_email
